In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

imputedDataset = pd.read_stata("/Users/burke/Documents/research/bpCog/mcm/mcm/data/fullyImputedDataset.dta")
imputedDataset.head()
noRecentUSInmigration = imputedDataset.loc[~imputedDataset.timeInUS.isin([1,2,3])]
earlyCrossSectionOfNonImmigrants = noRecentUSInmigration.loc[noRecentUSInmigration.year==2001]
# here we just need to throw out the very young people for comparisons
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.loc[earlyCrossSectionOfNonImmigrants.age > 18 + 2]

earlyCrossSectionOfNonImmigrants['strokeDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2015.values == 1) & (earlyCrossSectionOfNonImmigrants.ucod.values==2)), 1, 0)
earlyCrossSectionOfNonImmigrants['miDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2015.values == 1) & (earlyCrossSectionOfNonImmigrants.ucod.values==1)), 1, 0)
earlyCrossSectionOfNonImmigrants['cvDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.strokeDeath.values == 1) | (earlyCrossSectionOfNonImmigrants.miDeath.values==1)), 1, 0)
earlyCrossSectionOfNonImmigrants['nonCVDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2015 ==1) & (earlyCrossSectionOfNonImmigrants.strokeDeath.values == 0) & (earlyCrossSectionOfNonImmigrants.miDeath.values==0)), 1, 0)
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.sample(n=50000, weights=earlyCrossSectionOfNonImmigrants.WTINT2YR, replace=True)

In [2]:
import os
os.chdir("/Users/burke/Documents/research/bpCog/mcm")
from mcm.outcome import OutcomeType
from mcm.population import NHANESDirectSamplePopulation

pop = NHANESDirectSamplePopulation(n=100000, year=2001) # start the sim in 2001

In [3]:
popSurvival = pd.DataFrame({"index" : pop._people.index, "baseAge" : [person._age[0] for person in pop._people]} )
popSurvival['ageDecile'] = pd.qcut(popSurvival.baseAge, 10)
popSurvival['priorStroke'] = [person._stroke for person in pop._people]
popSurvival['priorMI'] = [person._mi for person in pop._people]


#rjn the sim forward for 14 years to sync up with NHANES population
for year in range(1,15):
    pop.advance(1)
    popSurvival['dead' + str(year)] = [person.is_dead() for person in pop._people]
    popSurvival['cvDeath' + str(year)] = [person.has_fatal_stroke() or person.has_fatal_mi() for person in pop._people]
    popSurvival['strokeDeath' + str(year)] = [person.has_fatal_stroke() for person in pop._people]
    popSurvival['miDeath' + str(year)] = [person.has_fatal_mi() for person in pop._people]
    popSurvival['nonCVDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==False) and (person.has_fatal_mi() == False) for person in pop._people]
    popSurvival['cvDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==True) or (person.has_fatal_mi() == True) for person in pop._people]
    

In [4]:
from mcm.outcome import OutcomeType
miIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.MI, 2000)
print(f"MI Incidence:  {miIncidence[0]}\nevent count: {miIncidence[1]}")

strokeIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.STROKE, 2000)
print(f"Stroke Incidence: {strokeIncidence[0]}\nevent count: {strokeIncidence[1]} ")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/burke/Documents/research/bpCog/mcm/mcm/population.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

MI Incidence:  259.59115733080324
event count: 3803.0
Stroke Incidence: 159.45628246909874
event count: 2563.0 


In [5]:
### Incideence by race
from mcm.race_ethnicity import NHANESRaceEthnicity
def isBlack(person):
    return person._raceEthnicity == NHANESRaceEthnicity.NON_HISPANIC_BLACK

def isBlackDF(person):
    return person.raceEthnicity == NHANESRaceEthnicity.NON_HISPANIC_BLACK

def isWhite(person):
    return person._raceEthnicity == NHANESRaceEthnicity.NON_HISPANIC_WHITE

def isWhiteDF(person):
    return person.raceEthnicity == NHANESRaceEthnicity.NON_HISPANIC_WHITE


whiteIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.STROKE, 2000, isWhite, isWhiteDF)
print(f"White Stroke Incidence:  {whiteIncidence[0]} \nevent count: {whiteIncidence[1]}")
blackIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.STROKE, 2000, isBlack, isBlackDF)
print(f"Black Stroke Incidence:  {blackIncidence[0]} \nevent count: {blackIncidence[1]}")


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/burke/Documents/research/bpCog/mcm/mcm/population.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

White Stroke Incidence:  131.1133045283657 
event count: 1733.0
Black Stroke Incidence:  252.28427077893113 
event count: 337.0


In [6]:
### Incideence by primvary vs. seconeary status...
def hasCVEvent(person):
    return person.has_stroke_prior_to_simulation() or person.has_mi_prior_to_simulation()

def hasCVEventDF(person):
    return person.miPriorToSim or person.strokePriorToSim

secondaryMIIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.MI, 2000, hasCVEvent,hasCVEventDF )
print(f"Secondary MI Incidence: {secondaryMIIncidence[0]} \nevent count: {secondaryMIIncidence[1]}")

### Incideence by primvary vs. seconeary status...
def noCVEvent(person):
    return (not person.has_stroke_prior_to_simulation()) and (not person.has_mi_prior_to_simulation())

def noCVEventDF(person):
    return (not person.miPriorToSim) and (not person.strokePriorToSim)

primaryMiIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.MI, 2000, noCVEvent, noCVEventDF)
print(f"Primary MI Incidence:  {primaryMiIncidence[0]} \nevent count: {primaryMiIncidence[1]}")

secondaryStrokeIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.STROKE, 2000, hasCVEvent,hasCVEventDF)
print(f"Secondary Stroke Incidence:  {secondaryStrokeIncidence[0]} \nevent count: {secondaryStrokeIncidence[1]}")

primaryStrokeIncidence = pop.calculate_mean_age_sex_standardized_incidence(OutcomeType.STROKE, 2000, noCVEvent,noCVEventDF)
print(f"Primary Stroke Incidence:  {primaryStrokeIncidence[0]} \nevent count: {primaryStrokeIncidence[1]}")



/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/burke/Documents/research/bpCog/mcm/mcm/population.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

Secondary MI Incidence: 185.82247357383284 
event count: 139.0
Primary MI Incidence:  273.92685381218433 
event count: 3664.0
Secondary Stroke Incidence:  120.60182428294002 
event count: 182.0
Primary Stroke Incidence:  169.0764467201248 
event count: 2381.0


In [22]:
df = pop.get_people_current_state_as_dataframe()
df['cvEvent'] = 0
df.loc[(df.miPriorToSim==1) | (df.strokePriorToSim==1), 'cvEvent']=1
df.cvEvent.value_counts()

0    95099
1     4901
Name: cvEvent, dtype: int64

In [16]:
### OK...evertyiung looks really great _ except for the difference bewteen primary and secondary prevention...
# the primary vs. secondary patients by self report in NAHNES look as you'd expect...but, the event rates don't hold up in the sim...where incidence goes in teh wrong direction.
# however...the event rates, aren't tha tbad. so, for MI 139 events/4901 patients = 2.8% event rate
# vs. 3.8% event rate kn those that don't have events.

# my guess is that this has something to do with event counting...that peopel that have an event prior to the sim aren't getting calcualted correctly because they ahve more than one event in their outcomes data structure


0    95099
1     4901
Name: cvEvent, dtype: int64

### MI incidence should be a around 200/100k (assuming that yeh underestiamtes because its in kaiser and primary prevention only)
— aroudn 175 in PhD, K. R., MD, A. S. G., MPH, T. K. L., RPh, D. M. B. P., PhD, A. E. C.-B., MD, S. P. F., et al. (2017). Trends in Incidence of Hospitalized Acute Myocardial Infarction in the Cardiovascular Research Network (CVRN). The American Journal of Medicine, 130(3), 317–327. http://doi.org/10.1016/j.amjmed.2016.09.014
— around 200 in Yeh, R. W., Sidney, S., Chandra, M., Sorel, M., Selby, J. V., & Go, A. S. (2010). Population trends in the incidence and outcomes of acute myocardial infarction. The New England Journal of Medicine, 362(23), 2155–2165. http://doi.org/10.1056/NEJMoa0908610

### Stroke incidence is harder to pin down — ranges from 130-300+
— aroudn 130 in Morgenstern, L. B., Smith, M. A., Sánchez, B. N., Brown, D. L., Zahuranec, D. B., Garcia, N., et al. (2013). Persistent ischemic stroke disparities despite declining incidence in Mexican Americans. Annals of Neurology, 74(6), 778–785. http://doi.org/10.1002/ana.23972
— around 200 in Kleindorfer, D. O., Khoury, J., Moomaw, C. J., Alwell, K., Woo, D., Flaherty, M. L., et al. (2010). Stroke Incidence Is Decreasing in Whites But Not in Blacks: A Population-Based Estimate of Temporal Trends in Stroke Incidence From the Greater Cincinnati/Northern Kentucky Stroke Study * Supplemental Material, 41(7), 1326–1331. http://doi.org/10.1161/STROKEAHA.109.575043
— appears to be much higher in REGARDS< (460) and then summarize rates from 100-480! Howard, V. J., Kleindorfer, D. O., Judd, S. E., Mcclure, L. A., Safford, M. M., Rhodes, J. D., et al. (2011). Disparities in stroke incidence contributing to disparities in stroke mortality. Annals of Neurology, 69(4), 619–627. http://doi.org/10.1002/ana.22385

### age standardized all cause mortality for the whole population...
- https://www.cdc.gov/nchs/data/nvsr/nvsr67/nvsr67_05.pdf
- in 2016, age standardized death rate is 728.8 deaths per 100,000, crude death rate is 849.3 per 100,000 population.




In [7]:
print(f"Overall mortality: {pop.calculate_mean_age_sex_standardized_mortality(2000)}")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/burke/Documents/research/bpCog/mcm/mcm/population.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

Overall mortality: 748.6942788422792


### Upshot — MI incidence is reasonable, perhaps a little high, Stroke incidence is reasonable, hard to judge given variability.
— Wouldn't make changes at this point, instead will wait until we have better partitioning models and better event moretality models from the combined cohorts

### Overall mortality is slightly overestimated and is something that we may need to recalibrate

In [23]:
# on UCOD 1 = MI, 2=stroke, 3 = other cause of death
earlyCrossSectionOfNonImmigrants.ucod.value_counts(normalize=True, sort=False)

0    0.88354
1    0.03866
2    0.00712
3    0.07068
Name: ucod, dtype: float64

In [24]:
print("NHANES Overall mortality %0.3f" % earlyCrossSectionOfNonImmigrants.diedBy2015.mean())
print("Sim aoverall mortality %0.3f" % popSurvival.dead14.mean())

NHANES Overall mortality 0.117
Sim aoverall mortality 0.131


In [25]:
print("NHANES CV mortality %0.3f" % earlyCrossSectionOfNonImmigrants.cvDeath.mean())
print("Sim CV mortality %0.3f" % popSurvival.cvDeath14.mean())

NHANES CV mortality 0.046
Sim CV mortality 0.010


In [26]:
print("NHANES non-CV mortality %0.3f" % earlyCrossSectionOfNonImmigrants.nonCVDeath.mean())
print("Sim non-CV mortality %0.3f" % popSurvival.nonCVDeath14.mean())

NHANES non-CV mortality 0.071
Sim non-CV mortality 0.121


In [27]:
print("NHANES Heart Diasease mortality %0.3f" % earlyCrossSectionOfNonImmigrants.miDeath.mean())
print("Sim MI mortality %0.3f" % popSurvival.miDeath14.mean())

NHANES Heart Diasease mortality 0.039
Sim MI mortality 0.005


In [28]:
print("NHANES Stroke mortality %0.3f" % earlyCrossSectionOfNonImmigrants.strokeDeath.mean())
print("Sim stroke mortality %0.3f" % popSurvival.strokeDeath14.mean())

NHANES Stroke mortality 0.007
Sim stroke mortality 0.004


### CV Mortality is the primary problem
<p>CV mortality is <b>much</b> highger in NHANES than in the simulation (4.6% vs. 0.8%), whereas non-CV mortality is pretty close (which makes sense since we derived the non-CV mortality omdel from NHANES). This is mostly due to massive (10-fold) underestimationof raw mortality due to MI (3.9% vs. 0.4%)
    
### Next steps
<ol>
    <li><b> Rerun non-CV mortality model in NHANES with updated data</b> (Done)</li>
    <li> <b>Run mortality incidence code on NHANES — are we wildly over in NHANES compared to CDC estimates?</b> (Done) </li>
    <li> Might have to run incidnce code on death cerrts — can we reproduce the CDC etimates (i.e is there a code problem in the incidence calc?)</li>
    <li> <b>Document the epi logic problem that is in the emali to Jeremy </b> (Done)</li>
    <li> <b>Play with the assumptions to see how far we could move things</b> (Done)</li>
    <li> Check incidende in subgroups by race/ethnicity and gender and for primary/secondary prevention</li>
    <li> <b>Is the story here about primary vs. secondary prevention? If secondaray events are much more common given primary events and they have much higher mortality, then this could close part of the gap... (Jeremy's email)</b> (Done)</li>
</ol>

### #2 Run mortality incidence code on NHANES — are we wildly over in NHANES compared to CDC estimates?



In [ ]:
earlyCrossSectionOfNonImmigrants['yearInSim'] = earlyCrossSectionOfNonImmigrants.monthsToDeath/12

# set years in sim to years until death amongst peopel that die.
ageStandard = pop.build_age_standard(2016)

for index, row in earlyCrossSectionOfNonImmigrants.iterrows():
    nhanesFemale = row['gender'] - 1
    if (row['ucod'] != 0):
        ageStandard.loc[((ageStandard['lowerAgeBound'] <= row['age']) & (ageStandard['upperAgeBound'] >= row['age']) & (ageStandard['female'] == nhanesFemale)),
                        'outcomeCount'] += 1
    ageStandard.loc[((ageStandard['lowerAgeBound'] <= row['age']) &
                     (ageStandard['upperAgeBound'] >= row['age']) &
                     (ageStandard['female'] == nhanesFemale)),
                    'simPersonYears'] += row['yearInSim']
ageStandard = pop.tabulate_age_specific_rates(ageStandard)
print (ageStandard['ageSpecificContribution'].sum())

### So, estiamted morality is higher in NHANES that in our overall population and consideralby higher than in daeth certificate data

My guess is that this is a calculation problem...NHANES and our sim are both coming up in the same neighborhood and its much higher than CDC...the next xtep would be to run teh same calc on teh CDC death certs, but that would be a time consuming step

### #3. Broadly, we have a problem with the epidemiology here.

1. <b>Age standardizd incidence rates of MI (mostly primary prevention?) are around 200/100k. (above)</b>
2. <b >Case fatality is around 10%</b> (same refs as above)
3. <b>THat implies that mortlaity should be around 20k/100k.</b> (And is very close to what the sim is doing)
4. <b>However, death certificate assessed morrality is also around 200/100k</b> (Sidney, S., Quesenberry, C. P., Jr, Jaffe, M. G., Sorel, M., Nguyen-Huynh, M. N., Kushi, L. H., et al. (2016). Recent Trends in Cardiovascular Mortality in the United States and Public Health Goals. JAMA Cardiology, 1(5), 594–6. http://doi.org/10.1001/jamacardio.2016.1326)


So...we have a 10-fold gap in mortality on our hands. Theories:
<ol>
<li><b>Part of this is the differnece between heart diasese and MI mortality.</b> Straight MI dx on death certs make the problem half as big 100k/200k (Underlying Cause of death 1999-2017.xls spreadsheet from CDC wonder(although, its not obvious that MI isn't the primary cause of mortality in a lot of the other "heart disease" dx, such as CHF...)</li>

<li><b>Death certificates over-report cardiovacular mortality</b>
<ul><li>Our paper on stroke AMCOD vs. UCOD compared to MI (which, agian likely understates the degree of overestimation  because many MIs will end up with heart disease deaths...)</li>
<li>Jack's paper linking death certs to admin data suggesting that correlaton is lousy and that "diseases of the heart" are plausibly over estriamted</li></ul></li>

<li><b> Primary vs. Secondary Preventino </b> Our current modles dont' increase rates for secondary events or case fatality...how much would we have to increase to get close to death certs? That seems like an assumption we can play with pretty easily</li>
<li><b> better mortality model - right now, old people aren't dying off enough with CV events</b></li>
</ol>

### 4. Play with the assumptions to see how far we could move things

First, let's double the risk for secondary prevention and double the case fatality rate in secondary prevention



In [ ]:
caseFatalityMultiplier = [1,2,3,4,5]
secondaryIncidenceMultiplier=[1,2,3,4,5,6,7]
results = []

for fatalityMult in caseFatalityMultiplier:
    for incidenceMult in secondaryIncidenceMultiplier:
        print("fatality {:d}, incidence {:d}".format(fatalityMult, incidenceMult))
        popSens = NHANESDirectSamplePopulation(n=3000, year=2001) 

        popSens._outcome_model_repository.secondary_mi_case_fatality = fatalityMult * popSens._outcome_model_repository.mi_case_fatality
        popSens._outcome_model_repository.secondary_stroke_case_fatality = fatalityMult * popSens._outcome_model_repository.stroke_case_fatality
        popSens._outcome_model_repository.secondary_prevention_multiplier = incidenceMult

        popSensDF = pd.DataFrame({"index" : popSens._people.index, "baseAge" : [person._age[0] for person in popSens._people]} )
        popSensDF['ageDecile'] = pd.qcut(popSensDF.baseAge, 10)
        for year in range(1,15):
            popSens.advance(1)
            popSensDF['dead' + str(year)] = [person.is_dead() for person in popSens._people]
            popSensDF['cvDeath' + str(year)] = [person.has_fatal_stroke() or person.has_fatal_mi() for person in popSens._people]
            popSensDF['strokeDeath' + str(year)] = [person.has_fatal_stroke() for person in popSens._people]
            popSensDF['miDeath' + str(year)] = [person.has_fatal_mi() for person in popSens._people]
            popSensDF['nonCVDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==False) and (person.has_fatal_mi() == False) for person in popSens._people]
            popSensDF['cvDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==True) or (person.has_fatal_mi() == True) for person in popSens._people]

        results.append((fatalityMult, 
                        incidenceMult, 
                        popSensDF.dead14.mean(), 
                        popSensDF.cvDeath14.mean(),
                        popSensDF.nonCVDeath14.mean(),
                        popSensDF.miDeath14.mean(),
                        popSensDF.strokeDeath14.mean()))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

cvMort= np.array([result[3] for result in results]).reshape(5,7)
plt.contourf(cvMort, levels=[0.0, 0.01, 0.015, 0.02, 0.025, 0.03,0.035, 0.04])
plt.xticks(np.arange(7), np.arange(1,8))
plt.yticks(np.arange(5), np.arange(1,6))
plt.xlabel('Secondary Risk Multiplier (ASCVD * x = risk in secondary prevention)')
plt.ylabel('Secondary Case Fatality Multiplier\n (Primary case fatality * y = \ncase fatality in secondary prevention)')
plt.colorbar()

#### So...to get a CV mortality rate that approximates what is measured in NHANES, we need to multiply the secondary prevention risk by 7! and the secondary case fatality risk by 5!

In [ ]:
earlyCrossSectionOfNonImmigrants['ageDecile'] = pd.qcut(earlyCrossSectionOfNonImmigrants.age, 10)

mortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['diedBy2015','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
mortalityByDecileSim = popSurvival[['dead10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(mortalityByDecileSim.index.codes+0.5, mortalityByDecileSim.dead10, width=0.5, label="sim")
nhanesBar = plt.bar(mortalityByDecileNHANES.index.codes, mortalityByDecileNHANES.diedBy2015, width=0.5, label="nhanes")
plt.legend()


In [ ]:
strokeMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['strokeDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
mortalityByDecileSim = popSurvival[['strokeDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(mortalityByDecileSim.index.codes+0.5, mortalityByDecileSim.strokeDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(mortalityByDecileNHANES.index.codes, strokeMortalityByDecileNHANES.strokeDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
miMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['miDeath','ageDecile']].groupby('ageDecile').mean()
miMortalityByDecileSim = popSurvival[['miDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(miMortalityByDecileSim.index.codes+0.5, miMortalityByDecileSim.miDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(miMortalityByDecileNHANES.index.codes, miMortalityByDecileNHANES.miDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
cvMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['cvDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
cvMortalityByDecileSim = popSurvival[['cvDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(cvMortalityByDecileSim.index.codes+0.5, cvMortalityByDecileSim.cvDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(cvMortalityByDecileNHANES.index.codes, cvMortalityByDecileNHANES.cvDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
nonCVMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['nonCVDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
nonCVMortalityByDecileSim = popSurvival[['nonCVDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

nonCVsimBar =plt.bar(nonCVMortalityByDecileSim.index.codes+0.5, nonCVMortalityByDecileSim.nonCVDeath10, width=0.5, label="sim non cv")
cvSimBar =plt.bar(cvMortalityByDecileSim.index.codes+0.5, cvMortalityByDecileSim.cvDeath10, width=0.5, label="sim cv", bottom=nonCVMortalityByDecileSim.nonCVDeath10)

nonCVnhanesBar = plt.bar(nonCVMortalityByDecileNHANES.index.codes, nonCVMortalityByDecileNHANES.nonCVDeath, width=0.5, label="nhanes nonCV")
cvnhanesBar = plt.bar(cvMortalityByDecileNHANES.index.codes, cvMortalityByDecileNHANES.cvDeath, width=0.5, bottom=nonCVMortalityByDecileNHANES.nonCVDeath, label="nhanes CV")
plt.legend()

### So, we have slight under prediction of mortality in the simulation. Is that because of the code or the data?

To test — we will rerun the simulation on the direct NHANES data that we use in the calibration...

In [ ]:
from mcm.gender import NHANESGender
from mcm.person import Person
from mcm.outcome import Outcome
from mcm.race_ethnicity import NHANESRaceEthnicity
from mcm.smoking_status import SmokingStatus
from mcm.cohort_risk_model_repository import CohortRiskModelRepository
from mcm.outcome_model_repository import OutcomeModelRepository

In [ ]:
import os 

def calcMortalityRunningFullModel(x):
        temp =  Person(
            age=x.age,
            gender=NHANESGender(int(x.gender)),
            raceEthnicity=NHANESRaceEthnicity(int(x.raceEthnicity)),
            sbp=x.meanSBP,
            dbp=x.meanDBP,
            a1c=x.a1c,
            hdl=x.hdl,
            ldl=x.ldl,
            trig=x.trig,
            totChol=x.tot_chol,
            bmi=x.bmi,
            smokingStatus=SmokingStatus(int(x.smokingStatus)),
            dfIndex=x.index,
            diedBy2011=x.diedBy2011)
        
        for year in range(1,11):
            if not temp.is_dead():
                temp.advance_year(riskModel, cohortModel)
        return temp.is_dead()

earlyCrossSectionOfNonImmigrants['estMortalityFromModel'] = earlyCrossSectionOfNonImmigrants.apply(calcMortalityRunningFullModel, axis=1)

In [ ]:
earlyCrossSectionOfNonImmigrants['ageDecile'] = pd.qcut(earlyCrossSectionOfNonImmigrants.age, 10)
output = earlyCrossSectionOfNonImmigrants.groupby('ageDecile')[['diedBy2011', 'estMortalityFromModel']].mean()
print(output)
plt.bar(output.index.codes+1, output.diedBy2011, width=0.5, label="actualDeath")
plt.bar(output.index.codes+1.5, output.estMortalityFromModel, width=0.5, label="model")

plt.legend()

### Ok...this looks quite good...on the same data, the under-prediction problem looks very modest

In [ ]:
earlyCrossSectionOfNonImmigrants.to_stata("/Users/burke/Documents/research/bpCog/nhanes/earlyCrossSectionOfNonImmigrants.dta")
earlyCrossSectionOfNonImmigrants.ageDecile.value_counts().sort_index()

In [ ]:
earlyCrossSectionOfNonImmigrants[['diedBy2011', 'estMortalityFromCox', 'estMortalityFromModel']].mean()

In [ ]:
# well...this is not easy to explain.
# for the most part the code-bsaed calculation looks similar, gut it under predicts mortaltiy substantially in the highest age decile
# this is a bit strange...but, not impossible. if you look at the within population validation, it seems that we under-predicted in the oldest cohort for our basee model as well
# i wonder if having a longer follow-up (by constraining to the early oohort might cause that problem...that should be straight-forward to check)

# the code based model is over-estmiaging mortalty thorughout.
# this is going to be harder to figure otu...
# possibilities: 1. the times aren't right...and the cohort model is getting exposed to more time than the comparators
# the time aggregation isn't right...the one year probabilities are summing to something greater than the whole

In [ ]:
earlyCrossSectionOfNonImmigrants.ucod.value_counts()

In [ ]:
# next steps — 1. see if we under-preditx mortality badly in the NHANES popualtion if we look at only the people that are enrolled early (and thus have long follow-up)